## 寻找市场上的强势股

In [ ]:
import a1chemy.data_source as data_source
import a1chemy.indicators as indicators
import a1chemy.plot.plotly_tools as plotly_tools
import pymongo
import operator
import pandas as pd
from tqdm.notebook import tqdm
import base64

# data 
def return_rate(t, day1, day2):
    length = t.raw_data.shape[0]
    if -length >= day1 or -length >= day2:
        return -100
    return 100*(t.close().iloc[day1] - t.close().iloc[day2])/t.close().iloc[day2]

def bias(t, a, b):
    #print("{} {}:{}, {}:{}".format(t.name, a, t.raw_data[a].iloc[-1], b, t.raw_data[b].iloc[-1]))
    return 100*(t.raw_data[a].iloc[-1] - t.raw_data[b].iloc[-1])/t.raw_data[b].iloc[-1]
def roc(ticks=None, extend={}):
    extend = extend if extend is not None else {}
    return {
        'name': ticks.name,
        'symbol': ticks.symbol,
        'CLOSE': "{:.2f}".format(ticks.close().iloc[-1]),
        '1D': return_rate(ticks, -1, -2),
        '5D': return_rate(ticks, -1, -6),
        '1M': return_rate(ticks, -1, -21),
        'C/S': bias(ticks, 'close', 'EMA20'),
        'S/M': bias(ticks, 'EMA20', 'EMA60'),
        'M/L': bias(ticks, 'EMA60', 'EMA120'),
        'thumbnail': ticks_image_html(ticks, 20),
        'shares': extend['shares'](exchange=ticks.exchange, symbol=ticks.symbol, count=20) if 'shares' in extend else None
    }
def percent_color(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

def ticks_image_html(ticks, count):
    t = plotly_tools.ticks_thumbnail(ticks=ticks, start=-count)
    encode_image = base64.b64encode(t.data).decode()
    return '<img src="data:image/png;base64,{}">'.format(encode_image)

def shares_image_html(exchange, symbol, count):
    try:
        fund = mongo_fund_client.find_one(exchange=exchange, symbol=symbol)
        f = plotly_tools.fund_amount_thumbnail(fund=fund, start=-count)
        encode_image = base64.b64encode(f.data).decode()
        return '<img src="data:image/png;base64,{}">'.format(encode_image)
    except Exception as e:
        print("get {} shares failed".format(symbol))
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)

def show(tag_id=None, extend=None):
    stocks_tree = mongo_tags_client.tree(id=tag_id)
    stocks = []
    for id, stock_tag in tqdm(stocks_tree.root.children.items()):
        try:
            exchange=stock_tag.values['exchange']
            symbol =stock_tag.values['symbol']
            ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
            ticks.raw_data['EMA20'] = indicators.ema(data=ticks.close(), day=20)
            ticks.raw_data['EMA60'] = indicators.ema(data=ticks.close(), day=60)
            ticks.raw_data['EMA120'] = indicators.ema(data=ticks.close(), day=120)
            stocks.append(ticks)
        except Exception as e:
            print("get data failed, exchange:{} symbol:{}".format(exchange, symbol))
    roc_list = []
    for stock_tick in stocks:
        ticks_data = roc(stock_tick, extend=extend)
        roc_list.append(ticks_data)
    sorted_roc_list = sorted(roc_list, key = operator.itemgetter('5D', 'C/S', 'S/M', 'M/L'), reverse=True)
    df=pd.DataFrame(sorted_roc_list)

    # df.style
    percent_list = list(df)[3:-2]
    return df.style.format("{:.2f}", subset=percent_list).applymap(percent_color, subset=percent_list)

In [ ]:
show('CASH_COW')

In [ ]:
show('TOP')

In [ ]:
show('ZH_ETF', extend={'shares': shares_image_html})

In [ ]:
show('ZH_STOCK')

In [ ]:
show('csi300')